In [1]:
import pandas as pd
import numpy as np
import git
from datetime import datetime as dt
from connect_to_mysql import Database
import networkx as nx
import matplotlib.pyplot as plt
import collections 
pd.set_option("display.max_columns", None)

In [2]:
repo = git.Repo(".", search_parent_directories=True).git.rev_parse("--show-toplevel")
#df = pd.read_csv(f"{repo}/data/cleaning/test_1651834681.csv")
#df = df[(df["state"] == "completed")]

In [3]:
Database.query_dataframe('rides', 'Show tables')

KeyboardInterrupt: 

In [ ]:
df_edges = Database.query_dataframe('stopconnection', "Select * From stopconnection")

In [ ]:
df_edges.dtypes

### Number of Drives 

In [ ]:
df.columns

In [ ]:
pd.to_datetime(df["scheduled_to"])[0].month

In [ ]:
frequencies = pd.DataFrame(df[(pd.to_datetime(df["scheduled_to"]).dt.month == 2) & (pd.to_datetime(df["scheduled_to"]).dt.year == 2022) ].groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()

frequencies

In [ ]:
frequencies["pickup_address"] = frequencies["pickup_address"].astype(str)
frequencies["dropoff_address"] = frequencies["dropoff_address"].astype(str)

In [ ]:
#df_edges = pd.read_excel(
#        f"{repo}/data/other/MoDstops+Preismodell.xlsx", sheet_name="Liste 2022"
#)

In [ ]:
df_edges.columns

In [ ]:
df_edges = df_edges[["id_start", "id_end"]]

In [ ]:
df_edges["id_start"] = df_edges["id_start"].astype(str)
df_edges["id_end"] = df_edges["id_end"].astype(str)

In [ ]:
edges = pd.merge(df_edges, frequencies,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "number_of_drives"]]
edges = edges[edges["number_of_drives"].isna() == False]
edges["number_of_drives"] = 28  / edges["number_of_drives"] 

In [ ]:
edges

In [ ]:
edges["number_of_drives"].fillna(0, inplace=True)

In [ ]:
#G = nx.from_pandas_edgelist(
#    edges,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "number_of_drives"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G)
#plt.show()

In [ ]:
#print(nx.shortest_path(G, source="10002", target="4007", weight = "number_of_drives"))
#print(nx.shortest_path_length(G, source="10002", target="4007", weight="number_of_drives", method='dijkstra'))

### Average Ride Time per drive  

In [ ]:
#df
#df.dtypes['ride_time']

In [ ]:
average = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean())
average.rename(columns = {"ride_time":"avg_ride_time"}, inplace = True)
average

In [ ]:
average["pickup_address"] = average["pickup_address"].astype(str)
average["dropoff_address"] = average["dropoff_address"].astype(str)

In [ ]:
edges2 = pd.merge(df_edges, average,  how='left', left_on=['id_start','id_end'], right_on = ['pickup_address','dropoff_address'])[["id_start", "id_end", "avg_ride_time"]]
edges2 = edges2[edges2["avg_ride_time"].isna() == False]
edges2

In [ ]:
#G1 = nx.from_pandas_edgelist(
#    edges2,
#    source = "id_start",
#    target = "id_end",
#    edge_attr= "avg_ride_time"
#)

In [ ]:
#plt.figure(3,figsize=(30,30)) 
#nx.draw(G1)
#plt.show()

In [ ]:
#print(nx.shortest_path(G1, source="1001", target="9015", weight = "avg_ride_time"))
#print(nx.shortest_path_length(G1, source="1001", target="9015", weight="avg_ride_time", method='dijkstra'))

### Original Function

In [ ]:
def get_shortest_ride(startpoint, endpoint, start_date, end_date):
    start_date = dt.strptime(start_date, "%m/%d/%Y")
    end_date = dt.strptime(end_date, "%m/%d/%Y")
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )

    path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
    shortest_time = nx.shortest_path_length(G, source=startpoint, target=endpoint, weight="avg_time_to_destination", method='dijkstra') 

    return (path, shortest_time)

### Set start and end date 

In [ ]:
df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
startdate = df.scheduled_to.min()
startdate = startdate.strftime('%m/%d/%Y')
start_date = dt.strptime(startdate, "%m/%d/%Y")
enddate = df.scheduled_to.max()
enddate = enddate.strftime('%m/%d/%Y')
end_date = dt.strptime(enddate, "%m/%d/%Y")

In [ ]:
#df['pickup_address'] = df['pickup_address'].map(str)
#df['dropoff_address'] = df['dropoff_address'].map(str)

### Drive DF

In [ ]:
days = (end_date - start_date).days + 1
drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
drives["waiting_time"] = days / drives["number_of_drives"]
drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

In [ ]:
drives.dtypes

### Graph

In [ ]:
G = nx.from_pandas_edgelist(
    drives,
    source = "pickup_address",
    target = "dropoff_address",
    edge_attr= "avg_time_to_destination"
)

In [ ]:
G.nodes

In [ ]:
def calculate_graph(start_date, end_date):
    days = (end_date - start_date).days + 1
    df = Database.query_dataframe('rides', "Select * From rides where state = 'completed'")
    df = df[(df["scheduled_to"] > start_date) & (df["scheduled_to"] < end_date)]
    
    drives = pd.DataFrame(df.groupby(["pickup_address", "dropoff_address"], group_keys=False).size().to_frame('number_of_drives')).reset_index()
    
    drives["waiting_time"] = days / drives["number_of_drives"]

    drives["avg_ride_time"] = df.groupby(["pickup_address", "dropoff_address"], as_index=False, group_keys=False)["ride_time"].mean()["ride_time"] / 60 / 60 / 24
    
    drives["avg_time_to_destination"] = drives["waiting_time"] + drives["avg_ride_time"]

    G = nx.from_pandas_edgelist(
        drives,
        source = "pickup_address",
        target = "dropoff_address",
        edge_attr= "avg_time_to_destination"
    )
    return G

In [ ]:
def get_shortest_ride(startpoint, endpoint, G):

    if startpoint not in G:
        return 'Not in G'
    elif endpoint not in G: 
        return 'Not in G'
    else:
        path = nx.shortest_path(G, source=startpoint, target=endpoint, weight = "avg_time_to_destination")
        return path

In [ ]:
df_test = pd.DataFrame({'Start':['1001', '1002', '1931391'], 'End': ['10004', '1008', '32432432']})
df_test["Start"] = df_test["Start"].astype(str).astype(int)
df_test["End"] = df_test["End"].astype(str).astype(int)

In [ ]:
G = calculate_graph(start_date, end_date)

In [ ]:
df_test['HotSpots'] = df_test.apply(lambda x: get_shortest_ride(x.Start, x.End, G), axis=1)

In [ ]:
df_edges['HotSpots'] = df_edges.apply(lambda x: get_shortest_ride(x.id_start, x.id_end, G), axis=1)

In [ ]:
df_edges

In [ ]:
hotSpots_list = list(df_edges['HotSpots'])

In [ ]:
flat_hotSpots_list = [item for sublist in hotSpots_list for item in sublist]

In [ ]:
flat_hotSpots_list

In [ ]:
counter=collections.Counter(flat_hotSpots_list)
print(counter.most_common(20))